In [31]:
import json

with open("dreeam/output_from_dreeam.json", "r") as file:
    dreeam_output = json.load(file)

with open("dreeam/dataset/docred/test.json", "r") as file:
    dreeam_input = json.load(file)

In [32]:
len(dreeam_output)

242

In [33]:
with open("dataset/meta/rel_info.json", "r") as file:
    rel_info = json.load(file)

In [34]:
relations = [{"input_idx": relation["title"].split()[-1], "h_idx": relation["h_idx"], "t_idx": relation["t_idx"], "r": relation["r"]} for relation in dreeam_output]
relations[10]

{'input_idx': '96', 'h_idx': 1, 't_idx': 0, 'r': 'P4'}

In [35]:
dreeam_input[0]

{'vertexSet': [[{'pos': [1, 3],
    'type': 'ORG',
    'sent_id': 0,
    'name': 'The Commission'},
   {'pos': [1, 3], 'type': 'ORG', 'sent_id': 1, 'name': 'The Commission'},
   {'pos': [1, 3], 'type': 'ORG', 'sent_id': 2, 'name': 'The Commission'}],
  [{'pos': [8, 9], 'type': 'DOC', 'sent_id': 0, 'name': 'Regulation'},
   {'pos': [8, 9], 'type': 'DOC', 'sent_id': 1, 'name': 'Regulation'},
   {'pos': [8, 9], 'type': 'DOC', 'sent_id': 2, 'name': 'Regulation'}],
  [{'pos': [14, 16],
    'type': 'ALG',
    'sent_id': 0,
    'name': 'artificial intelligence'},
   {'pos': [14, 16],
    'type': 'ALG',
    'sent_id': 1,
    'name': 'artificial intelligence'},
   {'pos': [14, 16],
    'type': 'ALG',
    'sent_id': 2,
    'name': 'artificial intelligence'}],
  [{'pos': [17, 19],
    'type': 'ALG',
    'sent_id': 0,
    'name': 'Artificial Intelligence'},
   {'pos': [17, 19],
    'type': 'ALG',
    'sent_id': 1,
    'name': 'Artificial Intelligence'},
   {'pos': [17, 19],
    'type': 'ALG',
    

In [36]:
triplets = []
for relation in relations:
    head = dreeam_input[int(relation["input_idx"])]["vertexSet"][relation["h_idx"]][0]["name"]
    tail = dreeam_input[int(relation["input_idx"])]["vertexSet"][relation["t_idx"]][0]["name"]
    head_cat = dreeam_input[int(relation["input_idx"])]["vertexSet"][relation["h_idx"]][0]["type"]
    tail_cat = dreeam_input[int(relation["input_idx"])]["vertexSet"][relation["t_idx"]][0]["type"]

    relation = rel_info[relation["r"]]
    
    if head == 'AI system':
        head = "AI systems"
        
    if tail == 'AI system':
        tail = "AI systems"
    
    triplets.append({"head": head, "head_cat": head_cat, "type": relation,"tail": tail, "tail_cat": tail_cat})
triplets

[{'head': 'law enforcement',
  'head_cat': 'ORG',
  'type': 'utlises',
  'tail': 'biometric identification',
  'tail_cat': 'ALG'},
 {'head': ' Regulatio',
  'head_cat': 'DOC',
  'type': 'require',
  'tail': ' complianc',
  'tail_cat': 'ETH'},
 {'head': 'AI',
  'head_cat': 'ALG',
  'type': 'should_be',
  'tail': 'trustworthy',
  'tail_cat': 'ETH'},
 {'head': 'AI systems',
  'head_cat': 'SYS',
  'type': 'require',
  'tail': 'data governance',
  'tail_cat': 'PRO'},
 {'head': 'AI systems',
  'head_cat': 'SYS',
  'type': 'require',
  'tail': 'compliance',
  'tail_cat': 'ETH'},
 {'head': 'stakeholders',
  'head_cat': 'PER',
  'type': 'are_central_to',
  'tail': 'ethical principles',
  'tail_cat': 'ETH'},
 {'head': 'systems',
  'head_cat': 'SYS',
  'type': 'utlises',
  'tail': ' biometric dat',
  'tail_cat': 'DAT'},
 {'head': 'AI systems',
  'head_cat': 'SYS',
  'type': 'involved_in',
  'tail': 'evaluate',
  'tail_cat': 'ACT'},
 {'head': 'natural persons',
  'head_cat': 'PER',
  'type': 'are_

In [38]:
with open("knowledge_graphs.jsonl", "r") as file:
    training_data = [json.loads(line) for line in file]
len(training_data)

408

In [39]:
combined_data = training_data + triplets
triplets = combined_data

In [42]:
len(triplets)

650

In [45]:
def find_matching_pairs(data):
    pairs = []
    seen = set()

    for i in range(len(data)):
        for j in range(i + 1, len(data)):
            dict1 = data[i]
            dict2 = data[j]

            # Check if head and tail match, but types are different
            if (dict1['head'], dict1['tail']) == (dict2['head'], dict2['tail']) and dict1['type'] != dict2['type']:
                # Create a key to ensure we don't find reverse duplicates
                key = tuple(sorted([dict1['head'], dict1['tail']]))

                if key not in seen:
                    pairs.append((dict1, dict2))
                    seen.add(key)

    return pairs

matching_pairs = find_matching_pairs(triplets)
matching_pairs

[({'head': 'Regulation',
   'head_cat': 'DOC',
   'type': 'governs_or_guides',
   'tail': 'AI system',
   'tail_cat': 'SYS'},
  {'head': 'Regulation',
   'head_cat': 'DOC',
   'type': 'has_implications_for',
   'tail': 'AI system',
   'tail_cat': 'SYS'}),
 ({'head': 'AI system',
   'head_cat': 'SYS',
   'type': 'contains',
   'tail': 'algorithms',
   'tail_cat': 'ALG'},
  {'head': 'AI system',
   'head_cat': 'SYS',
   'type': 'based_on',
   'tail': 'algorithms',
   'tail_cat': 'ALG'}),
 ({'head': 'AI system',
   'head_cat': 'SYS',
   'type': 'should_be',
   'tail': 'safety',
   'tail_cat': 'ETH'},
  {'head': 'AI system',
   'head_cat': 'SYS',
   'type': 'must_be',
   'tail': 'safety',
   'tail_cat': 'ETH'}),
 ({'head': 'AI system',
   'head_cat': 'SYS',
   'type': 'require',
   'tail': 'compliance',
   'tail_cat': 'ETH'},
  {'head': 'AI system',
   'head_cat': 'SYS',
   'type': 'is_affected_by',
   'tail': 'compliance',
   'tail_cat': 'ETH'}),
 ({'head': 'AI system',
   'head_cat': 'SY

In [44]:
len(find_matching_pairs(training_data))

22

In [43]:
with open("dreeam_outputs/dreeam_output_combined.jsonl", "w") as outfile:
    for data in triplets:
        json_line = json.dumps(data)
        outfile.write(json_line + "\n")

In [37]:
import networkx as nx
from pyvis.network import Network

G = nx.DiGraph()

for item in triplets:
    G.add_edge(item["head"], item["tail"], label=item["type"])
    
net = Network(notebook=False, height="750px", width="100%")
net.from_nx(G)

# Customize the visualization
net.show_buttons(filter_=['physics'])
net.toggle_physics(True)

# Save the visualization to a html file
net.save_graph("knowledge_graph/knowledge_graph.html")